# 🚀 YouTube Report Generator - Quick Demo

**Phase 1: Quick Demo Notebook**

## 📋 Overview
- **Purpose**: 샘플 데이터로 빠른 리포트 생성 데모
- **Execution Time**: < 10 minutes
- **Requirements**: CPU only, No API keys needed
- **Model**: BART-CNN (lightweight summarization)

## ✨ Features
1. Load sample video data from existing dataset
2. Generate video summary from title + description
3. Summarize audience reactions from comments
4. Create formatted markdown report

---

## 📦 Setup & Installation

Install required packages (if not already installed):

In [ ]:
# Install dependencies
!pip install -q transformers torch pandas

In [ ]:
# Import libraries
import json
import pandas as pd
from transformers import pipeline
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")

---

## 📂 Load Sample Data

Load the first video from our collected dataset:

In [ ]:
# Load sample dataset
DATA_PATH = 'full_dataset_20251013_215347.json'

try:
    with open(DATA_PATH, 'r', encoding='utf-8') as f:
        dataset = json.load(f)
    print(f"✅ Loaded {len(dataset)} videos from dataset")
except FileNotFoundError:
    print("⚠️ Dataset file not found. Please ensure 'full_dataset_20251013_215347.json' is in the same directory.")
    print("You can download it from the project repository.")
    raise

In [ ]:
# Select the first video for demo
sample_video = dataset[0]

# Extract key information
video_info = sample_video['video_info']
comments = sample_video['comments']

print("="*60)
print("📹 SAMPLE VIDEO INFORMATION")
print("="*60)
print(f"Title: {video_info['title']}")
print(f"Channel: {video_info['channel_title']}")
print(f"Views: {video_info['view_count']:,}")
print(f"Likes: {video_info['like_count']:,}")
print(f"Comments Count: {len(comments)}")
print(f"Category: {video_info['category_name']}")
print("="*60)

---

## 🤖 Initialize Summarization Model

Load BART-CNN model for text summarization (CPU-friendly):

In [ ]:
# Initialize summarization pipeline with BART-CNN
print("Loading BART-CNN summarization model...")
print("(This may take a minute on first run)")

summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",
    device=-1  # CPU mode
)

print("✅ Model loaded successfully!")

---

## 📝 Generate Video Summary

Summarize video content from title and description:

In [ ]:
def generate_video_summary(video_info, summarizer, max_length=150, min_length=50):
    """
    Generate video summary from title and description
    
    Args:
        video_info: Dictionary containing video information
        summarizer: Hugging Face summarization pipeline
        max_length: Maximum summary length
        min_length: Minimum summary length
    
    Returns:
        Summary text string
    """
    # Combine title and description
    title = video_info.get('title', '')
    description = video_info.get('description', '')
    
    # Create input text
    input_text = f"Title: {title}\n\nDescription: {description}"
    
    # Truncate if too long (BART max input is 1024 tokens)
    if len(input_text) > 3000:  # Rough character limit
        input_text = input_text[:3000] + "..."
    
    try:
        # Generate summary
        summary = summarizer(
            input_text,
            max_length=max_length,
            min_length=min_length,
            do_sample=False
        )
        return summary[0]['summary_text']
    except Exception as e:
        print(f"⚠️ Summarization error: {e}")
        return f"This video titled '{title}' was uploaded by {video_info.get('channel_title', 'Unknown')}."

# Generate video summary
print("Generating video summary...")
video_summary = generate_video_summary(video_info, summarizer)

print("\n" + "="*60)
print("📹 VIDEO SUMMARY")
print("="*60)
print(video_summary)
print("="*60)

---

## 💬 Generate Reaction Summary

Analyze and summarize audience reactions from comments:

In [ ]:
def generate_reaction_summary(comments, summarizer, top_n=20, max_length=100, min_length=30):
    """
    Generate audience reaction summary from top comments
    
    Args:
        comments: List of comment dictionaries
        summarizer: Hugging Face summarization pipeline
        top_n: Number of top comments to analyze
        max_length: Maximum summary length
        min_length: Minimum summary length
    
    Returns:
        Reaction summary text string
    """
    if not comments:
        return "No comments available for analysis."
    
    # Take top N comments (assuming they're already sorted by relevance/likes)
    top_comments = comments[:top_n]
    
    # Combine comment texts
    comments_text = "Viewer reactions: " + " ".join(
        [comment.get('text', '') for comment in top_comments if comment.get('text')]
    )
    
    # Truncate if too long
    if len(comments_text) > 2000:
        comments_text = comments_text[:2000] + "..."
    
    try:
        # Generate summary
        summary = summarizer(
            comments_text,
            max_length=max_length,
            min_length=min_length,
            do_sample=False
        )
        return summary[0]['summary_text']
    except Exception as e:
        print(f"⚠️ Comment summarization error: {e}")
        return f"Comments show various reactions from {len(comments)} viewers."

# Generate reaction summary
print(f"Analyzing top comments (sample size: {min(20, len(comments))})...")
reaction_summary = generate_reaction_summary(comments, summarizer)

print("\n" + "="*60)
print("💬 AUDIENCE REACTION SUMMARY")
print("="*60)
print(reaction_summary)
print("="*60)

---

## 📊 Basic Statistics

Calculate some basic engagement metrics:

In [ ]:
# Calculate engagement metrics
views = video_info['view_count']
likes = video_info['like_count']
num_comments = len(comments)

engagement_rate = (likes / views * 100) if views > 0 else 0
comment_rate = (num_comments / views * 100) if views > 0 else 0

print("\n" + "="*60)
print("📊 ENGAGEMENT METRICS")
print("="*60)
print(f"Views: {views:,}")
print(f"Likes: {likes:,}")
print(f"Comments: {num_comments:,}")
print(f"Like Rate: {engagement_rate:.2f}%")
print(f"Comment Rate: {comment_rate:.3f}%")
print("="*60)

---

## 📄 Generate Final Report

Create a formatted markdown report with all information:

In [ ]:
def generate_markdown_report(video_info, video_summary, reaction_summary, comments):
    """
    Generate formatted markdown report
    
    Args:
        video_info: Video metadata dictionary
        video_summary: Generated video summary text
        reaction_summary: Generated reaction summary text
        comments: List of comment dictionaries
    
    Returns:
        Markdown formatted report string
    """
    # Calculate metrics
    views = video_info['view_count']
    likes = video_info['like_count']
    num_comments = len(comments)
    engagement_rate = (likes / views * 100) if views > 0 else 0
    
    # Create report
    report = f"""# YouTube Video Report

## 📹 Video Information

**Title**: {video_info['title']}

**Channel**: {video_info['channel_title']}

**Category**: {video_info.get('category_name', 'Unknown')}

**Published**: {video_info['published_at']}

**Video ID**: `{video_info['video_id']}`

---

## 📊 Engagement Metrics

| Metric | Value |
|--------|-------|
| Views | {views:,} |
| Likes | {likes:,} |
| Comments | {num_comments:,} |
| Engagement Rate | {engagement_rate:.2f}% |

---

## 📝 Video Summary

{video_summary}

---

## 💬 Audience Reaction Summary

{reaction_summary}

---

## 🔍 Top Comments (Sample)

"""
    
    # Add top 5 comments as examples
    for i, comment in enumerate(comments[:5], 1):
        author = comment.get('author', 'Unknown')
        text = comment.get('text', '').replace('\n', ' ')[:200]  # Truncate long comments
        report += f"{i}. **{author}**: {text}...\n\n"
    
    report += """---

*Report generated by YouTube Report Generator - Quick Demo*
*Model: BART-CNN | Execution: CPU-only*
"""
    
    return report

# Generate the final report
final_report = generate_markdown_report(
    video_info=video_info,
    video_summary=video_summary,
    reaction_summary=reaction_summary,
    comments=comments
)

print("✅ Report generated successfully!")
print("\n" + "="*60)
print("📄 FINAL REPORT PREVIEW")
print("="*60)

In [ ]:
# Display the report
from IPython.display import Markdown, display

display(Markdown(final_report))

---

## 💾 Save Report to File

In [ ]:
# Save report to markdown file
output_filename = f"report_{video_info['video_id']}.md"

with open(output_filename, 'w', encoding='utf-8') as f:
    f.write(final_report)

print(f"✅ Report saved to: {output_filename}")

---

## 🎯 Summary & Next Steps

### What We Accomplished:
- ✅ Loaded sample video data from existing dataset
- ✅ Generated video summary using BART-CNN
- ✅ Analyzed and summarized audience reactions
- ✅ Created formatted markdown report
- ✅ Calculated engagement metrics

### Performance Notes:
- **Execution Time**: < 10 minutes (as required)
- **Resource Usage**: CPU-only (no GPU required)
- **API Keys**: None required (sample data)
- **Model**: BART-CNN (lightweight, suitable for demos)

### Next Steps (Phase 2):
1. **Full Pipeline Development**
   - Integrate YouTube API for real-time data collection
   - Upgrade to Llama-3.1-8B (4-bit) for better summaries
   - Implement Map-Reduce for long content
   - Add team models (category classification, sentiment analysis)

2. **Enhancements**
   - RAG system for context-aware summarization
   - Multi-language support
   - Advanced sentiment analysis
   - Automated evaluation metrics

---

## 📚 References

- **Model**: [facebook/bart-large-cnn](https://huggingface.co/facebook/bart-large-cnn)
- **Framework**: Hugging Face Transformers
- **Dataset**: Custom collected YouTube data (20 videos, 400 comments)

---

**Phase 1 Complete! 🎉**

This quick demo demonstrates the core functionality of our YouTube Report Generator. The full pipeline (Phase 2) will include more sophisticated models and real-time data collection.